##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Entrenamiento multitrabajador con Keras

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

## Visión general

Este tutorial demuestra cómo realizar un entrenamiento distribuido multitrabajador con un modelo Keras y la API `Model.fit` usando la API `tf.distribute.MultiWorkerMirroredStrategy`. Con la ayuda de esta estrategia, un modelo Keras que ha sido diseñado para ejecutarse en un único trabajador puede funcionar sin problemas en multitrabajador con cambios mínimos en el código.

Para aprender a usar la `MultiWorkerMirroredStrategy` con Keras y un bucle de entrenamiento personalizado, consulte [Bucle de entrenamiento personalizado con Keras y MultiWorkerMirroredStrategy](multi_worker_with_ctl.ipynb).

Este tutorial incluye un ejemplo mínimo de multitrabajador con dos trabajadores a modo de demostración.

### Elegir la estrategia adecuada

Antes de empezar, cerciórese de que `tf.distribute.MultiWorkerMirroredStrategy` es la opción adecuada para su(s) acelerador(es) y entrenamiento. Se trata de dos formas habituales de distribuir el entrenamiento con paralelismo de datos:

- *Entrenamiento síncrono*, en el que los pasos del entrenamiento se sincronizan entre los trabajadores y las réplicas, como `tf.distribute.MirroredStrategy`, `tf.distribute.TPUStrategy`, y `tf.distribute.MultiWorkerMirroredStrategy`. Todos los trabajadores se entrenan sobre diferentes porciones de datos de entrada de forma sincronizada, y agregando gradientes en cada paso.
- *Entrenamiento asíncrono*, en el que los pasos del entrenamiento no están estrictamente sincronizados, como `tf.distribute.experimental.ParameterServerStrategy`. Todos los trabajadores están entrenando de forma independiente sobre los datos de entrada y actualizando las variables de forma asíncrona.

Si lo que busca es un entrenamiento síncrono multitrabajador sin TPU, entonces necesita `tf.distribute.MultiWorkerMirroredStrategy`. Crea copias de todas las variables de las capas del modelo en cada dispositivo a través de todos los trabajadores. Usa `CollectiveOps`, una op de TensorFlow para la comunicación colectiva, para agregar gradientes y conserva las variables sincronizadas. Si está interesado, consulte la información sobre el parámetro `tf.distribute.experimental.CommunicationOptions` para ver las opciones de implementación de colectivos.

Consulte [Entrenamiento distribuido en TensorFlow](../../guide/distributed_training.ipynb) para ver un resumen de las API `tf.distribute.Strategy`.

## Preparación

Comience con algunas importaciones necesarias:

In [ ]:
import json
import os
import sys

Antes de importar TensorFlow, realice algunos cambios en el entorno:

- En una aplicación del mundo real, cada trabajador estaría en una máquina diferente. Para efectos de este tutorial, todos los trabajadores se ejecutarán en la máquina **this**. Así pues, desactive todas las GPU para evitar errores causados por todos los trabajadores que intentan usar la misma GPU.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

- Restablezca la variable de entorno `TF_CONFIG` (aprenderá más sobre esto más adelante):

In [ ]:
os.environ.pop('TF_CONFIG', None)

- Asegúrese de que el directorio actual está en la ruta de Python; esto permite al bloc de notas importar posteriormente los archivos escritos por `%%writefile`:


In [ ]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

Instale `tf-nightly`, ya que la frecuencia de guardado de puntos de verificación en un paso concreto con el argumento `save_freq` en `tf.keras.callbacks.BackupAndRestore` se introduce a partir de TensorFlow 2.10:

In [ ]:
!pip install tf-nightly

Finalmente, importe TensorFlow:

In [ ]:
import tensorflow as tf

### Definición del conjunto de datos y del modelo

A continuación, cree un archivo `mnist_setup.py` con una configuración sencilla del modelo y el conjunto de datos. Este archivo Python será usado por los procesos de los trabajadores en este tutorial:

In [ ]:
%%writefile mnist_setup.py

import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # The `x` arrays are in uint8 and have values in the [0, 255] range.
  # You need to convert them to float32 with values in the [0, 1] range.
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset

def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'])
  return model

### Entrenamiento de modelo en un solo trabajador

Pruebe a entrenar el modelo durante un pequeño número de épocas y observe los resultados de *un solo trabajador* para asegurarse de que todo funciona correctamente. Conforme avance el entrenamiento, la pérdida debería disminuir y la precisión debería aumentar.

In [ ]:
import mnist_setup

batch_size = 64
single_worker_dataset = mnist_setup.mnist_dataset(batch_size)
single_worker_model = mnist_setup.build_and_compile_cnn_model()
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)

## Configuración multitrabajador

Entremos ahora en el mundo del entrenamiento multitrabajador.

### Un cluster con trabajos y tareas

En TensorFlow, el entrenamiento distribuido implica un `'cluster'` con varios trabajos, y cada uno de los trabajos puede tener una o más `'task'`.

Necesitará la variable de entorno de configuración `TF_CONFIG` para el entrenamiento en múltiples máquinas, cada una de las cuales posiblemente tenga un rol diferente. `TF_CONFIG` es una cadena JSON empleada para especificar la configuración del clúster para cada trabajador que forme parte del clúster.

Hay dos componentes de una variable `TF_CONFIG`: `'cluster'` y `'task'`.

- Un `'cluster'` es el mismo para todos los trabajadores y da información sobre el cluster de entrenamiento, que es un diccionario formado por diferentes tipos de trabajos, como `'worker'` o `'chief'`.

    - En el entrenamiento multitrabajador con `tf.distribute.MultiWorkerMirroredStrategy`, suele haber un `'worker'` que asume más responsabilidades, como guardar un punto de verificación y escribir un archivo sumario para TensorBoard, además de lo que hace un `'worker'` normal. A este `'worker'` se le identifica como el "worker" jefe (con un nombre de trabajo `'chief'`).
    - Es habitual que el trabajador con `'index'` `0` sea el `'chief'`.

- Una `'task'` da información sobre la tarea actual y es diferente para cada trabajador. Especifica el `'type'` y el `'index'` de ese trabajador.

Below is an example configuration:

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

Tenga en cuenta que `tf_config` es sólo una variable local en Python. Para usarla para la configuración del entrenamiento, serialícela como JSON y colóquela en una variable de entorno `TF_CONFIG`.

In [ ]:
json.dumps(tf_config)

En el ejemplo de configuración anterior, se establece el `'type'` de tarea a `'worker'` y el `'index'` de la tarea en `0`. Por lo tanto, esta máquina es la *primera* trabajadora. Será designada como el trabajador `'chief'`.

Nota: Otras máquinas necesitarán tener la variable de entorno `TF_CONFIG` establecida también, y deberá tener el mismo diccionario `'cluster'`, pero diferentes `'type'` de tareas o `'index'` de tareas, dependiendo de los roles de esas máquinas.

En la práctica, usted crearía varios trabajadores en direcciones IP/puertos externos y configuraría una variable `TF_CONFIG` en cada trabajador según corresponda. Como ejemplo, este tutorial muestra cómo puede configurar una variable `TF_CONFIG` con dos trabajadores en un `localhost`:

- El `TF_CONFIG` del primer trabajador (`'chief'`) como se muestra arriba.
- Para el segundo trabajador, fijará `tf_config['task']['index']=1`

### Variables de entorno y subprocesos en blocs de notas

Los subprocesos heredan las variables de entorno de su padre. Así que si configura una variable de entorno en este proceso de bloc de notas Jupyter:

In [ ]:
os.environ['GREETINGS'] = 'Hello TensorFlow!'

... entonces puede acceder a la variable de entorno desde los subprocesos:

In [ ]:
%%bash
echo ${GREETINGS}

En la siguiente sección, usará este método para pasar el `TF_CONFIG` a los subprocesos de los trabajadores. En un escenario real nunca ejecutaría sus trabajos de esta forma; este tutorial sólo muestra cómo hacerlo con un ejemplo multitrabajador mínimo.

## Entrenar el modelo

Para entrenar el modelo, cree en primer lugar una instancia del modelo `tf.distribute.MultiWorkerMirroredStrategy`:

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

Nota: `TF_CONFIG` se parsea y los servidores GRPC de TensorFlow se inician en el momento en que se llama a `MultiWorkerMirroredStrategy`, por lo que la variable de entorno `TF_CONFIG` debe configurarse antes de crear una instancia `tf.distribute.Strategy`. Dado que `TF_CONFIG` aún no está establecida, la estrategia anterior es efectivamente un entrenamiento para un solo trabajador.

Con la integración de la API `tf.distribute.Strategy` en `tf.keras`, el único cambio que hará para distribuir el entrenamiento a múltiples trabajadores es acotar la construcción del modelo y la llamada `model.compile` dentro de `strategy.scope()`. El ámbito de la estrategia de distribución dicta cómo y dónde se crean las variables, y en el caso de `MultiWorkerMirroredStrategy`, las variables creadas son `MirroredVariable`, y se replican en cada uno de los trabajadores.


In [ ]:
with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()

Nota: Actualmente existe una limitación en `MultiWorkerMirroredStrategy` en la que las operaciones TensorFlow deben crearse después de crear la instancia de la estrategia. Si se encuentra con el error `RuntimeError: Collective ops must be configured at program startup`, pruebe a crear la instancia de `MultiWorkerMirroredStrategy` al principio del programa y ponga el código que pueda crear ops después de instanciar la estrategia.

Para ejecutar realmente con `MultiWorkerMirroredStrategy` necesitará ejecutar procesos worker y pasarles un `TF_CONFIG`.

Al igual que el archivo `mnist_setup.py` escrito anteriormente, aquí está el `main.py` que cada trabajador ejecutará:

In [ ]:
%%writefile main.py

import os
import json

import tensorflow as tf
import mnist_setup

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist_setup.mnist_dataset(global_batch_size)

with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

En el fragmento de código anterior, observe que `global_batch_size`, que se pasa a `Dataset.batch`, se configura en `per_worker_batch_size * num_workers`. Esto garantiza que cada trabajador procese lotes de `per_worker_batch_size` ejemplos independientemente del número de trabajadores.

El directorio actual contiene ahora ambos archivos Python:

In [ ]:
%%bash
ls *.py

Serialice el `TF_CONFIG` a JSON y añádalo a las variables de entorno:

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Ahora, puede iniciar un proceso trabajador que ejecutará el `main.py` y usará el `TF_CONFIG`:

In [ ]:
# first kill any previous runs
%killbgscripts

In [ ]:
%%bash --bg
python main.py &> job_0.log

Hay que tener en cuenta algunas cosas sobre el comando anterior:

1. Éste usa el `%%bash` que es una ["magia" de bloc de notas](https://ipython.readthedocs.io/en/stable/interactive/magics.html) para ejecutar algunos comandos bash.
2. Usa la bandera `--bg` para ejecutar el proceso `bash` en segundo plano, porque este trabajador no terminará. Espera a todos los trabajadores antes de iniciarse.

El proceso trabajador en segundo plano no imprimirá la salida en este bloc de notas, por lo que el `&>` redirige su salida a un archivo para que pueda inspeccionar lo sucedido en un archivo de registro más adelante.

Por lo tanto, espere unos segundos a que se inicie el proceso:

In [ ]:
import time
time.sleep(10)

Ahora, inspeccione lo que se ha volcado en el archivo de registro del trabajador hasta el momento:

In [ ]:
%%bash
cat job_0.log

La última línea del archivo de registro debería decir: `Started server with target: grpc://localhost:12345`. El primer trabajador ya está listo y está esperando a que todos los demás trabajadores estén listos para continuar.

Así que actualice el `tf_config` para que el proceso del segundo trabajador retome el trabajo:

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Inicie el segundo trabajador. Esto iniciará el entrenamiento, ya que todos los trabajadores están activos (por lo que no es necesario poner este proceso en segundo plano):

In [ ]:
%%bash
python main.py

Si se vuelven a revisar los registros escritos por el primer trabajador, se sabrá que participó en el entrenamiento de ese modelo:

In [ ]:
%%bash
cat job_0.log

Nota: Esto puede correr más lento que la ejecución de prueba al principio de este tutorial porque ejecutar múltiples trabajadores en una sola máquina sólo añade sobrecarga. La meta aquí no es mejorar el tiempo de entrenamiento sino dar un ejemplo de entrenamiento multitrabajador.


In [ ]:
# Delete the `TF_CONFIG`, and kill any background tasks so they don't affect the next section.
os.environ.pop('TF_CONFIG', None)
%killbgscripts

## Entrenamiento multitrabajador a fondo


Por el momento, ha aprendido a realizar una configuración multitrabajador básica. El resto del tutorial repasa en detalle otros factores que pueden ser útiles o importantes para casos de uso reales.

### Fragmentación de conjuntos de datos

En el entrenamiento multitrabajador, *es necesario fragmentar el conjunto de datos* para garantizar la convergencia y el rendimiento.

El ejemplo de la sección anterior se basa en el autosharding predeterminado proporcionado por la API `tf.distribute.Strategy`. Puede controlar el fragmentado estableciendo la `tf.data.experimental.AutoShardPolicy` del `tf.data.experimental.DistributeOptions`.

Para saber más sobre *auto fragmentación*, consulte la [Guía de entrada distribuida](https://www.tensorflow.org/tutorials/distribute/input#sharding).

Aquí tiene un ejemplo rápido de cómo desactivar la auto fragmentación, para que cada réplica procese cada ejemplo (*no recomendado*):


In [ ]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

global_batch_size = 64
multi_worker_dataset = mnist_setup.mnist_dataset(batch_size=64)
dataset_no_auto_shard = multi_worker_dataset.with_options(options)

### Evaluación

Si pasa también el `validation_data` a `Model.fit`, alternará entre entrenamiento y evaluación para cada época. El trabajo de evaluación se distribuye entre el mismo grupo de trabajadores, y sus resultados se agregan y están disponibles para todos los trabajadores.

Parecido al entrenamiento, el conjunto de datos de validación se auto fragmenta a nivel de archivo. Debe configurar un tamaño de lote global en el conjunto de datos de validación y configurar los `validation_steps`.

Se recomienda utilizar un conjunto de datos repetido (llamando a `tf.data.Dataset.repeat`) para la evaluación.

Otra posibilidad es crear otra tarea que lea periódicamente los puntos de verificación y ejecute la evaluación. Esto es lo que hace un Estimator. Pero no se recomienda esta forma de realizar la evaluación, por lo que se omiten sus detalles.

### Rendimiento

Para mejorar el rendimiento del entrenamiento multitrabajador, puede probar lo siguiente:

- `tf.distribute.MultiWorkerMirroredStrategy` provee múltiples [implementaciones de comunicación de colectivos](https://www.tensorflow.org/api_docs/python/tf/distribute/experimental/CommunicationImplementation):

    - `RING` implementa colectivos en anillos usando gRPC como capa de comunicación entre hosts.
    - `NCCL` usa la [Librería de Comunicación Colectiva de NVIDIA](https://developer.nvidia.com/nccl) para implementar los colectivos.
    - `AUTO` deja la elección al runtime.

    La mejor elección para implementar colectivos depende del número y el tipo de GPU y de las interconexiones de red del cluster. Para anular la elección automática, especifique el parámetro `communication_options` del constructor de `MultiWorkerMirroredStrategy`. Por ejemplo:

    ```python
    communication_options=tf.distribute.experimental.CommunicationOptions(implementation=tf.distribute.experimental.CommunicationImplementation.NCCL)
    ```

- Haga cast de las variables a `tf.float` si es posible:

    - El modelo oficial de ResNet incluye [un ejemplo](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466) de cómo hacerlo.

### Tolerancia a fallos

En el entrenamiento síncrono, el cluster fallaría si uno de los trabajadores falla y no existe ningún mecanismo de recuperación de fallos.

Usar Keras con `tf.distribute.Strategy` ofrece la ventaja de la tolerancia a fallos cuando los trabajadores mueren o son inestables por algún otro motivo. Puede hacerlo conservando el estado de entrenamiento en el sistema de archivos distribuido de su elección, de forma que al reiniciar la instancia que falló o se adelantó previamente, se recupere el estado de entrenamiento.

Cuando un trabajador deja de estar disponible, otros trabajadores fallarán (posiblemente tras un tiempo de espera). En tales casos, es necesario reiniciar el trabajador no disponible, así como otros trabajadores que hayan fallado.

Nota: Anteriormente, la retrollamada `ModelCheckpoint` servía de mecanismo para restaurar el estado de entrenamiento tras un reinicio por fallo de un trabajo para el entrenamiento multitrabajador. El equipo de TensorFlow está añadiendo una nueva retrollamada [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w), que añade también el soporte al entrenamiento de un solo trabajador para una experiencia consistente, y ha eliminado la funcionalidad de tolerancia a fallos de la retrollamada `ModelCheckpoint` existente. A partir de ahora, las aplicaciones que dependan de este comportamiento deberán migrar a la nueva retrollamada `BackupAndRestore`.

#### La retrollamada `ModelCheckpoint`

La retrollamada `ModelCheckpoint` ya no ofrece la funcionalidad de tolerancia a fallos, así que use en su lugar la retrollamada [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w).

La retrollamada `ModelCheckpoint` aún puede usarse para guardar puntos de verificación. Sin embargo, si el entrenamiento se interrumpió o finalizó con éxito, para continuar el entrenamiento a partir del punto de verificación, el usuario es responsable de cargar el modelo manualmente.

De forma opcional, los usuarios pueden elegir guardar y restaurar el modelo/las ponderaciones fuera de la retrollamada `ModelCheckpoint`.

### Guardar y cargar modelos

Para guardar su modelo usando `model.save` o `tf.saved_model.save`, es necesario que la dirección de guardado sea diferente para cada trabajador.

- Para los trabajadores que no sean jefes, tendrá que guardar el modelo en un directorio temporal.
- Para el jefe, tendrá que guardar en el directorio de modelos proporcionado.

Los directorios temporales del trabajador deben ser únicos para evitar errores resultantes de que varios trabajadores intenten escribir en la misma ubicación.

El modelo guardado en todos los directorios es idéntico, y por lo general sólo hay que consultar el modelo guardado por el jefe para restaurarlo o usarlo.

Debe tener alguna lógica de limpieza que borre los directorios temporales creados por los trabajadores una vez que su entrenamiento se haya llenado.

La razón para guardar en el jefe y en los trabajadores al mismo tiempo es que podría estar agregando variables durante el punto de verificación, lo que requiere que tanto el jefe como los trabajadores participen en el protocolo de comunicación allreduce. Por otro lado, dejar que el jefe y los trabajadores guarden en el mismo directorio del modelo resultará en errores debido a la contención.

Usando la `MultiWorkerMirroredStrategy`, el programa se ejecuta en cada trabajador, y para saber si el trabajador actual es el jefe, aprovecha el objeto resolutor del cluster que tiene atributos `task_type` y `task_id`:

- `task_type` le indica cuál es el trabajo actual (por ejemplo, `'worker'`).
- `task_id` le indica el identificador del trabajador.
- El trabajador con `task_id == 0` es designado como trabajador jefe.

En el fragmento de código siguiente, la función `write_filepath` indica la ruta del archivo a escribir, que depende del `task_id` del trabajador:

- Para el trabajador jefe (con `task_id == 0`), escribe en la ruta del archivo original.
- Para otros trabajadores, crea un directorio temporal (`temp_dir`) con el `task_id` en la ruta del directorio para escribir en él:

In [ ]:
model_path = '/tmp/keras-model'

def _is_chief(task_type, task_id):
  # Note: there are two possible `TF_CONFIG` configurations.
  #   1) In addition to `worker` tasks, a `chief` task type is use;
  #      in this case, this function should be modified to
  #      `return task_type == 'chief'`.
  #   2) Only `worker` task type is used; in this case, worker 0 is
  #      regarded as the chief. The implementation demonstrated here
  #      is for this case.
  # For the purpose of this Colab section, the `task_type` is `None` case
  # is added because it is effectively run with only a single worker.
  return (task_type == 'worker' and task_id == 0) or task_type is None

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

task_type, task_id = (strategy.cluster_resolver.task_type,
                      strategy.cluster_resolver.task_id)
write_model_path = write_filepath(model_path, task_type, task_id)

Con eso, ya está listo para guardar:

Obsoleto: Para los objetos Keras, se recomienda usar el nuevo formato de alto nivel `.keras` y `tf.keras.Model.export`, como se demuestra en la guía [aquí](https://www.tensorflow.org/guide/keras/save_and_serialize). El formato SavedModel de bajo nivel sigue siendo compatible con el código existente.

In [ ]:
multi_worker_model.save(write_model_path)

Como ya se ha descrito, más adelante el modelo sólo debe cargarse desde la ruta del archivo en el que guardó el trabajador jefe. Por lo tanto, elimine los temporales que hayan guardado los trabajadores no jefes:

In [ ]:
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(os.path.dirname(write_model_path))

Ahora, cuando sea el momento de cargar, use la práctica API `tf.keras.models.load_model`, y continúe con el trabajo posterior.

Aquí, supongamos que sólo se usa un único trabajador para cargar y continuar el entrenamiento, en cuyo caso no se llama a `tf.keras.models.load_model` dentro de otro `strategy.scope()` (tenga en cuenta que `strategy = tf.distribute.MultiWorkerMirroredStrategy()`, como se definió anteriormente):

In [ ]:
loaded_model = tf.keras.models.load_model(model_path)

# Now that the model is restored, and can continue with the training.
loaded_model.fit(single_worker_dataset, epochs=2, steps_per_epoch=20)

### Guardado y restauración del punto de verificación

Por otro lado, el punto de verificación le permite guardar las ponderaciones de su modelo y restaurarlas sin tener que guardar todo el modelo.

Aquí, creará un `tf.train.Checkpoint` que realice el seguimiento del modelo, que será administrado por el `tf.train.CheckpointManager`, de forma que sólo se conserve el último punto de verificación:

In [ ]:
checkpoint_dir = '/tmp/ckpt'

checkpoint = tf.train.Checkpoint(model=multi_worker_model)
write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

Una vez configurado el `CheckpointManager`, ya está listo para guardar y eliminar los puntos de verificación que hayan guardado los trabajadores no jefes:

In [ ]:
checkpoint_manager.save()
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(write_checkpoint_dir)

Ahora, cuando necesite restaurar el modelo, puede encontrar el último punto de verificación guardado usando la práctica función `tf.train.latest_checkpoint`. Tras restaurar el punto de verificación, puede continuar con el entrenamiento.

In [ ]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint.restore(latest_checkpoint)
multi_worker_model.fit(multi_worker_dataset, epochs=2, steps_per_epoch=20)

#### La retrollamada `BackupAndRestore`

La retrollamada `tf.keras.callbacks.BackupAndRestore` ofrece la funcionalidad de tolerancia a fallos haciendo una copia de seguridad del modelo y del estado actual del entrenamiento en un archivo temporal de puntos de verificación bajo el argumento `backup_dir` a `BackupAndRestore`.

Nota: En Tensorflow 2.9, se realiza una copia de seguridad del modelo actual y del estado de entrenamiento en los límites de las épocas. En la versión `tf-nightly` y a partir de TensorFlow 2.10, la retrollamada `BackupAndRestore` puede realizar una copia de seguridad del modelo y del estado de entrenamiento en los límites de época o de paso. `BackupAndRestore` acepta un argumento opcional `save_freq`. `save_freq` acepta ya sea `'epoch'` o un valor `int`. Si `save_freq` se configura como `'epoch'` se realiza una copia de seguridad del modelo después de cada época. Si `save_freq` se configura como valor entero superior a `0`, se realiza una copia de seguridad del modelo después de cada `save_freq` número de lotes.

Una vez que los trabajos se interrumpen y se reinician, la retrollamada `BackupAndRestore` restaura el último punto de verificación, y puede continuar el entrenamiento desde el principio de la época y el paso en los que se guardó por última vez el estado de entrenamiento.

Para usarlo, facilite una instancia de `tf.keras.callbacks.BackupAndRestore` en la llamada `Model.fit`.

Con `MultiWorkerMirroredStrategy`, si un trabajador es interrumpido, todo el cluster hará una pausa hasta que el trabajador interrumpido vuelva a arrancar. Los demás trabajadores también se reiniciarán y el trabajador interrumpido volverá a unirse al clúster. Entonces, cada trabajador leerá el archivo de punto de verificación que se guardó previamente y retomará su estado anterior, permitiendo así que el clúster vuelva a entrar en sincronía. A continuación, continuará el entrenamiento. El estado del iterador del conjunto de datos distribuido se reiniciará y no se restaurará.

La retrollamada `BackupAndRestore` usa el `CheckpointManager` para guardar y restaurar el estado de entrenamiento, lo que genera un archivo llamado checkpoint que hace un seguimiento de los puntos de verificación existentes junto con el más reciente. Por este motivo, `backup_dir` no debe usarse para almacenar otros puntos de verificación a fin de evitar la colisión de nombres.

Actualmente, la retrollamada `BackupAndRestore` admite el entrenamiento de un solo trabajador sin estrategia (`MirroredStrategy`) y el entrenamiento multitrabajador con `MultiWorkerMirroredStrategy`.

He aquí dos ejemplos tanto para el entrenamiento de multitrabajadores como para el de un solo trabajador:

In [ ]:
# Multi-worker training with `MultiWorkerMirroredStrategy`
# and the `BackupAndRestore` callback. The training state 
# is backed up at epoch boundaries by default.

callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup')]
with strategy.scope():
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)

Si el argumento `save_freq` de la retrollamada `BackupAndRestore` se configura como `'epoch'`, se realiza una copia de seguridad del modelo después de cada época.

In [ ]:
# The training state is backed up at epoch boundaries because `save_freq` is
# set to `epoch`.

callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup')]
with strategy.scope():
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)


Nota: El siguiente bloque de código usa funciones que sólo disponibles en `tf-nightly` hasta que se publique Tensorflow 2.10.

Si el argumento `save_freq` de la retrollamada `BackupAndRestore` se configura en un valor entero superior a `0`, se hace una copia de seguridad del modelo después de cada `save_freq` número de lotes.

In [ ]:
# The training state is backed up at every 30 steps because `save_freq` is set
# to an integer value of `30`.

callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup', save_freq=30)]
with strategy.scope():
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)

Si examina el directorio de `backup_dir` que especificó en `BackupAndRestore`, puede que note algunos archivos de puntos de verificación generados temporalmente. Esos archivos son necesarios para recuperar las instancias perdidas anteriormente, y serán eliminados por la librería al final de `Model.fit` al salir con éxito de su entrenamiento.

Nota: Actualmente la retrollamada `BackupAndRestore` sólo soporta eager mode. En modo grafo, considere usar `Model.save`/`tf.saved_model.save` y `tf.keras.models.load_model` para guardar y recuperar modelos, respectivamente, como se describe en la sección *Guardar y cargar modelos* anterior, y proporcionando `initial_epoch` en `Model.fit` durante el entrenamiento.

## Recursos adicionales

1. La guía [Entrenamiento distribuido en TensorFlow](../../guide/distributed_training.ipynb) proporciona una visión general de las estrategias de distribución disponibles.
2. El tutorial [Bucle de entrenamiento personalizado con Keras y MultiWorkerMirroredStrategy](multi_worker_with_ctl.ipynb) muestra cómo usar la `MultiWorkerMirroredStrategy` con Keras y un bucle de entrenamiento personalizado.
3. Consulte los [modelos oficiales](https://github.com/tensorflow/models/tree/master/official), muchos de los cuales pueden configurarse para ejecutar múltiples estrategias de distribución.
4. La guía [Un mejor rendimiento con tf.function](../../guide/function.ipynb) ofrece información sobre otras estrategias y herramientas, como el [Perfilador TensorFlow](../../guide/profiler.md) que puede usar para optimizar el rendimiento de sus modelos TensorFlow.